In [8]:
#%load_ext autoreload
#%autoreload 2
import torch

from torch import nn

from bronze_age.models.bronze_age import BronzeAgeGNN

from bronze_age.config import BronzeConfig, DatasetEnum, LayerTypeBronze, LayerType, NetworkType, AggregationMode, NonLinearity

from train_bronze import get_config_for_dataset, get_dataset, LightningModel

In [6]:
dataset_enum = DatasetEnum.SIMPLE_SATURATION

config = get_config_for_dataset(dataset_enum)

dataset = get_dataset(config)

In [10]:
gnn = BronzeAgeGNN(dataset.num_node_features, dataset.num_classes, config)
model = LightningModel.load_from_checkpoint("lightning_logs/15/03/25 19:06 Simple_Saturation/version_0/checkpoints/epoch=519-step=520.ckpt", model=gnn)

In [15]:
(model.model(dataset[0].x, dataset[0].edge_index)[0].argmax(dim=-1) == dataset[0].y).float().mean()

tensor(0.9965)

In [17]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(dataset, batch_size=128, shuffle=False)
model_dt = model.model.train_concept_model(train_loader)


  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | layer_dict | ModuleDict | 402 K  | train
--------------------------------------------------
402 K     Trainable params
0         Non-trainable params
402 K     Total params
1.609     Total estimated model params size (MB)
46        Modules in train mode
0         Modules in eval mode


In [18]:
(model_dt(dataset[0].x, dataset[0].edge_index)[0].argmax(dim=-1) == dataset[0].y).float().mean()

tensor(1.)

In [21]:
explanations = model_dt(dataset[0].x, dataset[0].edge_index, return_explanation=True)[2]

In [22]:
explanations

{'input': [{'class': 'y_0', 'explanation': '~c_1', 'count': 1980},
  {'class': 'y_1', 'explanation': 'c_2', 'count': 10},
  {'class': 'y_2', 'explanation': 'c_1', 'count': 10}],
 'StoneAgeLayer-0': [{'class': 'y_0',
   'explanation': 's_2_count>5',
   'count': 88},
  {'class': 'y_0', 'explanation': 's_1_count>2 & s_2_count>5', 'count': 242},
  {'class': 'y_0', 'explanation': 's_1_count>2', 'count': 66},
  {'class': 'y_0', 'explanation': 's_2_count>5 & s_2_count>8', 'count': 132},
  {'class': 'y_0', 'explanation': 's_2_count>8', 'count': 264},
  {'class': 'y_1', 'explanation': 's_2_count>2', 'count': 110},
  {'class': 'y_2', 'explanation': '~s_2_count>0 & ~s_2_count>5', 'count': 20},
  {'class': 'y_2',
   'explanation': '~s_1_count>7 & ~s_2_count>2 & ~s_2_count>5 & ~s_2_count>6',
   'count': 176},
  {'class': 'y_2', 'explanation': 's_1_count>5', 'count': 550},
  {'class': 'y_2',
   'explanation': '~s_1_count>7 & ~s_2_count>5 & ~s_2_count>6',
   'count': 220},
  {'class': 'y_2', 'explana

In [ ]:
# c_1: blue
# c_2: red

# input: y_0 = white
# input: y_1 : red
# input: y_2 : blue
# 
# 
# layer_0: blue > 5 or red > 2 or red > 8 
# layer_1: blue count > 2 